# Matrix Generation and Plot

Python code used to generate $ cos \Delta \Phi $ matrix used in simulation for this paper: 

Lai, A.; Saleem, Q.; Macdonald, P.M. Centerband-Only-Detection-of-Exchange 31P nuclear magnetic resonance and phospholipid lateral diffusion: Theory, simulation and experiment. _Physical Chemistry Chemical Physics_ __2015__, 17, 25160-25171.

## Theory
This code was written to calculate the complete powder average signal of the CODEX solid-state NMR experiment. The normalized CODEX signal intensity is given by the equation

$$ 			\frac{S(t_{m},\delta Mt_{r})}{S_{0}(t_{z},\delta Mt_{r})}=\big \langle cos \big( |\Phi_{2}|-| \Phi_{1}| \big) \big \rangle $$

where $ \Phi_{i} $ is the phase angle acquired during a given CODEX recoupling period containing M 180 degree pulses, while the angled brackets represent the powder average over all spins. The phase acquired during a given CODEX recoupling period is

$$ |\Phi_{i}|= \Big |M\int_{0}^{t_{r}/2} \omega_{i}(t)dt \Big | $$

where the precession frequency $\omega_{i}(t)dt$ becomes time-dependent under magic angle spinning. The equation for calculating the time-dependent precession frequency due to a rotation at an angular frequency $(\omega_{r})$ for an axially symmetric chemical shielding tensor

$$ \omega_{i}(t)=\frac{\delta}{2}\Bigg[sin^{2}\beta_{i}cos2(\alpha_{i}+\omega_{r}t)-\sqrt{2}sin2\beta_{i}cos(\alpha_{i}+\omega_{r}t)\Bigg] $$

where $\beta_{i}$ and $\alpha_{i}$ are polar and azimuthal angles respectively, defining the orientation of the unique axis $\omega_{z}$ within the reference frame of the spinning axis, as shown below, with $\alpha = 0$ placed arbitrarily in the plane defined by the spinning axis and the magnetic field.

<div style="width: 350px">![img/CODEXGeometry.png](img/CODEXGeometry.png)</div>




Thus the $ cos \Delta \Phi $ matrix can be calculated for all $^{31}$P nuclei of phospholipids on a spherical vesicle (LUV)

## Python Code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.clf() #Clears current plot

""" Input Variables Here """
N = 3        # Number of 180 degree pulses
CSA = 34871  # CSA of lipid in units of radians per second
vr = 6500    # Spin speed in Hz

def Calculate(b,th,a,p):
    s = np.sin(a)*np.sin(2*b)*(1-np.sin(th)*np.sin(th)*(1+np.cos(p)*np.cos(p)))                
    t = np.cos(a)*np.sin(b)*np.sin(2*p)*np.sin(th)*np.sin(th)                
    u = np.sin(a)*np.cos(2*b)*np.cos(p)*np.sin(2*th)                
    v = np.cos(a)*np.cos(b)*np.sin(p)*np.sin(2*th)                
    w = (s-t+u+v)-(np.sin(a)*np.sin(2*b))                
    x = np.cos(abs(w*Z))    
    return x

tr = 1/vr    # Rotor period in seconds
wr = vr * 2 * np.pi     # Rotor speed in radians per second
Z = (N * CSA * np.sqrt(2))/wr     #Phase prefactor

""" Generate Initial Angles """
BetaI = np.arange(0,182,2)
AlphaI = np.arange(0,182,2)
ThetaJ = np.arange(0,182,2)
PhiJ = np.arange(0,182,2)

BetaI = BetaI * (np.pi/180)  # Convert angles to radians
AlphaI = AlphaI * (np.pi/180)
ThetaJ = ThetaJ * (np.pi/180)
PhiJ = PhiJ * (np.pi/180)

x = []
y = []

CosDeltaPhiMatrix = np.empty([len(BetaI),len(ThetaJ)])

values = np.array([[(i,j) for i in range(11)] for j in range(22)]).reshape(-1,2)
vecCalc = np.vectorize(Calculate)
for indexB, Bi in np.ndenumerate(BetaI):    
    for indexT, Tk in np.ndenumerate(ThetaJ):        
        output = ([Calculate(Bi,Tk,value[0],value[1]) for value in values])
        CosDeltaPhiMatrix[indexT,indexB] = np.mean(output)

<div style="width: 350px">![img/CosDeltaPhiMatrix.png](img/CosDeltaPhiMatrix.png)</div>
